In [1]:
%pylab inline
import matplotlib.pyplot as plt


Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
from scipy.stats import ttest_rel
import copy
import pickle
import pandas as pd
import copy

from sklearn.preprocessing import MinMaxScaler


import numpy as np
from numpy import linalg as LA
import scipy
from scipy.sparse import csr_matrix
from sklearn import preprocessing

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel


from scipy.stats.stats import spearmanr
from scipy.stats.stats import pearsonr

from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA

In [3]:
we_vector_size = 300
glove_we_model_dir = "wordembeddings/glove/wikipedia/wiki-english-20171001/GloVe/"


In [4]:
glove_we_vocabs = []
glove_we_vectors = []
glove_we_biases = []
glove_we_ctx_vectors = []
glove_we_ctx_biases = []
with open(glove_we_model_dir+'/vectors.txt', 'r') as fr:
    for line in fr:
        vals = line.strip().split()
        glove_we_vocabs.append(vals[0])
        glove_we_vectors.append([float(x) for x in vals[1:1+we_vector_size]])
        glove_we_biases.append([float(x) for x in vals[1+we_vector_size:2+we_vector_size]][0])
        glove_we_ctx_vectors.append([float(x) for x in vals[2+we_vector_size:2+2*we_vector_size]])
        glove_we_ctx_biases.append([float(x) for x in vals[2+2*we_vector_size:]][0])
        
        
glove_we_vocabs = np.array(glove_we_vocabs)
glove_we_vectors = np.array(glove_we_vectors)
glove_we_biases = np.array(glove_we_biases)
glove_we_ctx_vectors = np.array(glove_we_ctx_vectors)
glove_we_ctx_biases = np.array(glove_we_ctx_biases)

print ('loading done!')

loading done!


In [5]:
glove_we_vocabs_indextowords = {}
for v_i, v in enumerate(glove_we_vocabs):
    glove_we_vocabs_indextowords[v] = v_i

In [6]:
def get_recGlove_vecs(words):
    recGlove_vecs = {}
    for word in words:
        _idx = glove_we_vocabs_indextowords[word]
        _vec = glove_we_vectors[_idx]
        
        recGlove_vecs[word] = np.dot(glove_we_ctx_vectors, _vec)

    return recGlove_vecs

get_recGlove_vecs(['book'])

{'book': array([ 5.52318689,  5.43175299,  4.46455256, ...,  0.00788603,
        -1.31117989, -1.35755048])}

In [7]:
def Cosine(vec1, vec2):
    return 1 - scipy.spatial.distance.cosine(vec1, vec2)

def NegDifNorm(vec1, vec2):
    return -np.linalg.norm((vec1/np.linalg.norm(vec1)) - (vec2/np.linalg.norm(vec2)))

def meanCosine(vec, set_vecs):
    return np.mean([Cosine(vec, _set_vec) for _set_vec in set_vecs])


def get_recGlove_vecs_similarity(word_from, words_to):
    recGlove_vecs_from = get_recGlove_vecs([word_from])
    recGlove_vecs_to = get_recGlove_vecs(words_to)
    
    sims_recGlove={}
    for word_to in words_to:
        if word_to in glove_we_vocabs:
            sims_recGlove[word_to] = Cosine(recGlove_vecs_from[word_from], recGlove_vecs_to[word_to])
    
    return sims_recGlove

get_recGlove_vecs_similarity('book', ['book', 'library'])

{'book': 1.0, 'library': 0.7135090843891218}

## Gender Biasing Quantification

In [8]:
job_tuples=[]
with open('resources/wordlist_occupations.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        if _tuple[0] in glove_we_vocabs:
            job_tuples.append(_tuple)
print (len(job_tuples))
job_recGlove_vecs = get_recGlove_vecs([x[0] for x in job_tuples])


497


In [9]:
wordpairs_debiasing_direction = []
with open('resources/wordpairs_direction.txt') as fr:
    for l in fr:
        wordpairs_debiasing_direction.append(l.strip().split(','))
print (len(wordpairs_debiasing_direction))



28


In [10]:
Glove_genderword_tuples = []

with open('resources/wordlist_genderspecific.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        if _tuple[0] in glove_we_vocabs:
            _idx = glove_we_vocabs_indextowords[_tuple[0]]
            Glove_genderword_tuples.append((_tuple[0], _tuple[1], _idx))
print (len(Glove_genderword_tuples))
genderword_recGlove_vecs = get_recGlove_vecs([x[0] for x in Glove_genderword_tuples])


56


In [11]:
fem_Glove_vecs = []
masc_Glove_vecs = []
for _tuple in Glove_genderword_tuples:
    if _tuple[1] == 'f':
        fem_Glove_vecs.append(glove_we_vectors[glove_we_vocabs_indextowords[_tuple[0]]])
    elif _tuple[1] == 'm':
        masc_Glove_vecs.append(glove_we_vectors[glove_we_vocabs_indextowords[_tuple[0]]])
    
fem_Glove_centroid_vec = np.mean(np.array(fem_Glove_vecs), axis=0)
masc_Glove_centroid_vec = np.mean(np.array(masc_Glove_vecs), axis=0)



In [12]:
fem_recGlove_vecs = []
masc_recGlove_vecs = []
for _tuple in Glove_genderword_tuples:
    _recGlove_vecs = get_recGlove_vecs([_tuple[0]])
    _recGlove_vec = _recGlove_vecs[_tuple[0]]
    if _tuple[1] == 'f':
        fem_recGlove_vecs.append(_recGlove_vec)
    elif _tuple[1] == 'm':
        masc_recGlove_vecs.append(_recGlove_vec)
    
fem_recGlove_centroid_vec = np.mean(np.array(fem_recGlove_vecs), axis=0)
masc_recGlove_centroid_vec = np.mean(np.array(masc_recGlove_vecs), axis=0)


In [13]:
#Gender direction

_matrix_Glove = []
_matrix_recGlove = []
for a, b in wordpairs_debiasing_direction:
    _a_Glove_vec = glove_we_vectors[glove_we_vocabs_indextowords[a]]
    _b_Glove_vec = glove_we_vectors[glove_we_vocabs_indextowords[b]]
    _a_Glove_vec_norm = _a_Glove_vec/np.linalg.norm(_a_Glove_vec)
    _b_Glove_vec_norm = _b_Glove_vec/np.linalg.norm(_b_Glove_vec)
    _center_Glove_vec = (_a_Glove_vec_norm + _b_Glove_vec_norm)/2
    _matrix_Glove.append(_a_Glove_vec_norm - _center_Glove_vec)
    _matrix_Glove.append(_b_Glove_vec_norm - _center_Glove_vec)
    
    _recGlove_vecs = get_recGlove_vecs([a, b])
    _a_recGlove_vec = _recGlove_vecs[a]
    _b_recGlove_vec = _recGlove_vecs[b]
    _a_recGlove_vec_norm = _a_recGlove_vec/np.linalg.norm(_a_recGlove_vec)
    _b_recGlove_vec_norm = _b_recGlove_vec/np.linalg.norm(_b_recGlove_vec)
    _center_recGlove_vec = (_a_recGlove_vec_norm + _b_recGlove_vec_norm)/2
    _matrix_recGlove.append(_a_recGlove_vec_norm - _center_recGlove_vec)
    _matrix_recGlove.append(_b_recGlove_vec_norm - _center_recGlove_vec)

_matrix_Glove = np.array(_matrix_Glove)
_matrix_recGlove = np.array(_matrix_recGlove)
pca = PCA(n_components = 10)
gendir_Glove_vec = pca.fit(_matrix_Glove).components_[0]
gendir_recGlove_vec = pca.fit(_matrix_recGlove).components_[0]




In [14]:
Glove_fem_exp_feature_vec = np.zeros(len(glove_we_vocabs))
Glove_masc_exp_feature_vec = np.zeros(len(glove_we_vocabs))
for _tuple in Glove_genderword_tuples:
    if _tuple[1] == 'f':
        Glove_fem_exp_feature_vec[_tuple[2]] = 1
    elif _tuple[1] == 'm':
        Glove_masc_exp_feature_vec[_tuple[2]] = 1


In [15]:
print (len([x for x in Glove_genderword_tuples if x[1]=='f']), len([x for x in Glove_genderword_tuples if x[1]=='m']))

28 28


### calculating bias

In [16]:
job_recGlove_1st_mean_bias = []
for job_tuple in job_tuples:
    _vec = job_recGlove_vecs[job_tuple[0]]
    job_recGlove_1st_mean_bias.append((job_tuple[0], np.dot(_vec, Glove_fem_exp_feature_vec)/np.sum(Glove_fem_exp_feature_vec) - np.dot(_vec, Glove_masc_exp_feature_vec)/np.sum(Glove_masc_exp_feature_vec),
                                       np.dot(_vec, Glove_fem_exp_feature_vec)/np.sum(Glove_fem_exp_feature_vec), 
                                       np.dot(_vec, Glove_masc_exp_feature_vec)/np.sum(Glove_masc_exp_feature_vec), job_tuple[1]))
    
job_recGlove_1st_mean_bias.sort(key=lambda x: x[1])



In [ ]:
job_recGlove_2nd_gendir_bias = []
for job_tuple in job_tuples:
    _vec = job_recGlove_vecs[job_tuple[0]]
    job_recGlove_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_recGlove_vec), 0.0, 0.0, job_tuple[1]))
    
job_recGlove_2nd_gendir_bias.sort(key=lambda x: x[1])



In [30]:
job_recGlove_2nd_centroid_bias = []
for job_tuple in job_tuples:
    _vec = job_recGlove_vecs[job_tuple[0]]
    job_recGlove_2nd_centroid_bias.append((job_tuple[0], Cosine(_vec, fem_recGlove_centroid_vec) - Cosine(_vec, masc_recGlove_centroid_vec),
                              Cosine(_vec, fem_recGlove_centroid_vec), Cosine(_vec, masc_recGlove_centroid_vec), job_tuple[1]))
    
job_recGlove_2nd_centroid_bias.sort(key=lambda x: x[1])


In [20]:


job_recGlove_2nd_mean_bias = []
for job_tuple in job_tuples:
    _vec = job_recGlove_vecs[job_tuple[0]]
    job_recGlove_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, fem_recGlove_vecs) - meanCosine(_vec, masc_recGlove_vecs),
                              meanCosine(_vec, fem_recGlove_vecs), meanCosine(_vec, masc_recGlove_vecs), job_tuple[1]))
    
job_recGlove_2nd_mean_bias.sort(key=lambda x: x[1])





In [18]:
job_Glove_2nd_gendir_bias = []
for job_tuple in job_tuples:
    _vec = glove_we_vectors[glove_we_vocabs_indextowords[job_tuple[0]]]
    job_Glove_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_Glove_vec), 0.0, 0.0, job_tuple[1]))
    
job_Glove_2nd_gendir_bias.sort(key=lambda x: x[1])




In [19]:


job_Glove_2nd_mean_bias = []
for job_tuple in job_tuples:
    _vec = glove_we_vectors[glove_we_vocabs_indextowords[job_tuple[0]]]
    job_Glove_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, fem_Glove_vecs) - meanCosine(_vec, masc_Glove_vecs),
                              meanCosine(_vec, fem_Glove_vecs), meanCosine(_vec, masc_Glove_vecs), job_tuple[1]))
    
job_Glove_2nd_mean_bias.sort(key=lambda x: x[1])




In [31]:
job_Glove_2nd_centroid_bias = []
for job_tuple in job_tuples:
    _vec = glove_we_vectors[glove_we_vocabs_indextowords[job_tuple[0]]]
    job_Glove_2nd_centroid_bias.append((job_tuple[0], Cosine(_vec, fem_Glove_centroid_vec) - Cosine(_vec, masc_Glove_centroid_vec),
                              Cosine(_vec, fem_Glove_centroid_vec), Cosine(_vec, masc_Glove_centroid_vec), job_tuple[1]))
    
job_Glove_2nd_centroid_bias.sort(key=lambda x: x[1])



## Correlation to Occupation Statistics

In [32]:

winobias_stats=[]
with open('resources/occupations_stats_winobias.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        winobias_stats.append((_tuple[0], float(_tuple[1])))
print (len(winobias_stats))

census_stats=[]
with open('resources/occupations_stats_census.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        census_stats.append((_tuple[0], float(_tuple[1])))
print (len(census_stats))


40
96


In [33]:
from scipy.stats.stats import spearmanr
from scipy.stats.stats import pearsonr

def calc_correlations(_stats_jobs, _stats_bias):
    job_bias_data = {'job_recGlove_1st_mean_bias':job_recGlove_1st_mean_bias,
                     'job_recGlove_2nd_mean_bias':job_recGlove_2nd_mean_bias,
                     'job_recGlove_2nd_centroid_bias':job_recGlove_2nd_centroid_bias,
                     'job_recGlove_2nd_gendir_bias':job_recGlove_2nd_gendir_bias,
                     'job_Glove_2nd_mean_bias':job_Glove_2nd_mean_bias,
                     'job_Glove_2nd_centroid_bias':job_Glove_2nd_centroid_bias,
                     'job_Glove_2nd_gendir_bias':job_Glove_2nd_gendir_bias,
                    }
    labor_bias = {}
    for _data_key in job_bias_data:
        labor_bias[_data_key] = []
    _idx = 1
    for _job in _stats_jobs:
        for _data_key in job_bias_data:
            labor_bias[_data_key].append([x for x in job_bias_data[_data_key] if x[0]==_job][0][_idx])
        
    results = {'spearman':{}, 'pearson':{}}
    print ('Spearman')
    for _data_key in job_bias_data:
        _text = _data_key.replace('job_', '').replace('_bias', '')
        results['spearman'][_text] = abs(round(spearmanr(labor_bias[_data_key], _stats_bias).correlation, 2))
        print ("%20s %0.2f" % (_text, results['spearman'][_text]))
    print ()
    print ('Pearson')
    for _data_key in job_bias_data:
        _text = _data_key.replace('job_', '').replace('_bias', '')
        results['pearson'][_text] = abs(round(pearsonr(labor_bias[_data_key], _stats_bias)[0], 2))
        print ("%20s %0.2f" % (_text, results['pearson'][_text]))
    print ()
    
    return results
    
results = {}

print ("WinoBias")
print ("----------")
results['winobias'] = calc_correlations([x[0] for x in winobias_stats], [x[1] for x in winobias_stats])
print ("**********")
#calc_correlations([x[0] for x in winobias_stats], [np.log((x[1]/100)/(1-(x[1]/100))) for x in winobias_stats])
#print ("**********")
#print ()
print ("U.S. Census Database")
#calc_correlations([x[0] for x in census_stats], [np.log(x[1]/(1-x[1])) for x in census_stats])
#print ("**********")
#print ()
results['census'] = calc_correlations([x[0] for x in census_stats], [x[1] for x in census_stats])
print ("**********")
print ()





WinoBias
----------
Spearman
   recGlove_1st_mean 0.56
   recGlove_2nd_mean 0.48
recGlove_2nd_centroid 0.48
 recGlove_2nd_gendir 0.35
      Glove_2nd_mean 0.60
  Glove_2nd_centroid 0.58
    Glove_2nd_gendir 0.53

Pearson
   recGlove_1st_mean 0.57
   recGlove_2nd_mean 0.51
recGlove_2nd_centroid 0.52
 recGlove_2nd_gendir 0.34
      Glove_2nd_mean 0.61
  Glove_2nd_centroid 0.60
    Glove_2nd_gendir 0.56

**********
U.S. Census Database
Spearman
   recGlove_1st_mean 0.42
   recGlove_2nd_mean 0.16
recGlove_2nd_centroid 0.24
 recGlove_2nd_gendir 0.07
      Glove_2nd_mean 0.39
  Glove_2nd_centroid 0.39
    Glove_2nd_gendir 0.34

Pearson
   recGlove_1st_mean 0.52
   recGlove_2nd_mean 0.24
recGlove_2nd_centroid 0.35
 recGlove_2nd_gendir 0.12
      Glove_2nd_mean 0.51
  Glove_2nd_centroid 0.51
    Glove_2nd_gendir 0.46

**********



In [34]:
for x in ['Glove_2nd_gendir','Glove_2nd_mean','Glove_2nd_centroid',
          'recGlove_2nd_gendir','recGlove_2nd_mean', 'recGlove_2nd_centroid', 
          'recGlove_1st_mean', ]:
    print ("%0.2f & %0.2f & %0.2f & %0.2f" % (results['winobias']['spearman'][x],
                                             results['winobias']['pearson'][x],
                                             results['census']['spearman'][x],
                                             results['census']['pearson'][x]))



0.53 & 0.56 & 0.34 & 0.46
0.60 & 0.61 & 0.39 & 0.51
0.58 & 0.60 & 0.39 & 0.51
0.35 & 0.34 & 0.07 & 0.12
0.48 & 0.51 & 0.16 & 0.24
0.48 & 0.52 & 0.24 & 0.35
0.56 & 0.57 & 0.42 & 0.52
